In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip3 install --upgrade --quiet google-cloud-aiplatform[langchain] openai
! pip3 install --upgrade --quiet langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.3 MB/s eta 0:00:00


In [3]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

In [2]:
PROJECT_ID = "gen-lang-client-0539303742"
LOCATION = "us-central1"
BUCKET_NAME = "ml_Llama"
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [3]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [4]:
# Chat completions API
import openai
from google.auth import default, transport
from langchain import PromptTemplate
# Build
from langchain_openai import ChatOpenAI

In [5]:
def reAuth():
  credentials, _ = default()
  auth_request = transport.requests.Request()
  credentials.refresh(auth_request)

  MODEL_LOCATION = "us-central1"
  base_url=f"https://{MODEL_LOCATION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{MODEL_LOCATION}/endpoints/openapi/chat/completions?"
  client = openai.OpenAI (
          base_url=base_url,
          api_key=credentials.token
          )
  return client

In [6]:
client = reAuth()

In [7]:
MODEL_ID = "meta/llama-3.1-405b-instruct-maas"

In [8]:
temperature = 0.6
max_tokens = 8192
top_p = 0.95
stream = False

In [9]:
testFilePath = "/content/drive/MyDrive/test.pkl"

In [10]:
import pandas as pd

testDf = pd.read_pickle(testFilePath)

In [11]:
testDf.head()

,content,comments,category,label,content_emotion,comments_emotion,emotion_gap,style_feature
0,"Born and raised in the Philippines, Lea Salong...",https://soundcloud.com/opeyemi-sanni/freak … ...,gossipcop,0,"[4.0, 23.5, 0.0, 0.0, 18.5, 0.0, 35.25, 4.0, 9...","[0.0, 0.3333333333333333, 0.0, 0.3333333333333...","[4.0, 23.166666666666668, 0.0, -0.333333333333...","[0.0, 0.005847953216374269, 0.0, 0.0, 0.0, 0.0..."
1,"We love all the Disney Princesses, but we're a...",“@eonline: All of the Disney Princesses' Wardr...,gossipcop,0,"[11.25, 16.25, 6.25, 12.25, 17.75, 14.25, 37.2...","[0.1875, 0.28125, 0.125, 0.0625, 0.34375, 0.25...","[11.0625, 15.96875, 6.125, 12.1875, 17.40625, ...","[0.022624434389140267, 0.011695906432748536, 0..."
2,Do you feel it in your fingers? Do you feel it...,@eonline It's Rick Grimes!!!!! @eonline hi @e...,gossipcop,0,"[28.5, 36.25, 14.0, 28.75, 50.75, 44.25, 79.75...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[28.5, 36.25, 14.0, 28.75, 50.75, 44.25, 79.75...","[0.04072398190045249, 0.011695906432748536, 0...."
3,"At the time of Prince’s death, his Paisley Par...",,gossipcop,0,"[7.0, 11.5, 4.0, 13.0, 5.5, 16.25, 8.5, 10.5, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.0, 11.5, 4.0, 13.0, 5.5, 16.25, 8.5, 10.5, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.008298755186721992..."
4,"Dr. Calliope Iphegenia ""Callie"" Towers\n\natte...",,gossipcop,0,"[5.0, 8.0, 3.0, 9.0, 7.0, 12.0, 10.5, 8.0, 9.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[5.0, 8.0, 3.0, 9.0, 7.0, 12.0, 10.5, 8.0, 9.0...","[0.0, 0.0, 0.005272407732864675, 0.0, 0.0, 0.0..."


In [12]:
toDrop = ["content_emotion", "comments_emotion", "emotion_gap", "style_feature"]
testDf = testDf.drop(toDrop, axis=1, inplace=False)

In [13]:
testDf.head()

,content,comments,category,label
0,"Born and raised in the Philippines, Lea Salong...",https://soundcloud.com/opeyemi-sanni/freak … ...,gossipcop,0
1,"We love all the Disney Princesses, but we're a...",“@eonline: All of the Disney Princesses' Wardr...,gossipcop,0
2,Do you feel it in your fingers? Do you feel it...,@eonline It's Rick Grimes!!!!! @eonline hi @e...,gossipcop,0
3,"At the time of Prince’s death, his Paisley Par...",,gossipcop,0
4,"Dr. Calliope Iphegenia ""Callie"" Towers\n\natte...",,gossipcop,0


In [14]:
nfdf = testDf[testDf['label'] == 0] #True
fdf = testDf[testDf['label'] == 1] #False (Fake)

In [15]:
print(len(nfdf), len(fdf))

4406 1341


In [16]:
fdfNoLabel = fdf.drop(columns=['label'], inplace=False)

In [17]:
fdfNoLabel.head()

,content,comments,category
3357,"Earlier this year, the buzz around Megyn Kelly...",,gossipcop
3358,For the first time since his involvement in a ...,Terribly sad. I hope ppl pay attention to what...,gossipcop
3359,Angelina Jolie has her eye on a new guy — hott...,,gossipcop
3360,Kate Hudson has dated some of Hollywood’s A-li...,,gossipcop
3361,Does Blac Chyna have a bun in the oven?! The 2...,,gossipcop


In [18]:
pip install json-repair

In [21]:
response = client.chat.completions.create(model=MODEL_ID, messages=[{"role": "user", "content": "Hello! what's up"}],temperature=temperature,max_tokens=max_tokens,top_p=top_p,)
print(response.choices[0].message.content)

Hello! Not much, just here and ready to chat. How about you? How's your day going so far?


In [22]:
sysInstr = '''
You are an expert at crafting news articles that are challenging for advanced fake news detection models (e.g., MDFEND, RoBERTa) to classify.
Your task is to subtly modify the content and comments of news articles in JSON format to retain their misleading intent while evading detection.
Use the following strategies:
- Rephrase content to obscure clear domain-specific features, making it difficult to associate the article with a specific domain (e.g., blend political and entertainment language).
- Introduce hypothetical or opinion-based framing (e.g., 'It is reported...' or 'Experts suggest...') while maintaining plausibility.
- Mimic credible sources by using professional, journalistic tones and neutral or balanced language.
- Add plausible but unverifiable details to increase complexity and reduce specific cues (e.g., avoid explicit dates, locations, or statistics).
- if there are any names of people mentioned in the news, keep them as they are.
- Disrupt sentence patterns by introducing diverse structures, conflicting tones, or unrelated details to dilute semantic coherence.
- Modify comments to include conflicting interpretations or perspectives (e.g., supportive, critical, and neutral), creating a mix of sentiments.
- Where applicable, create cross-domain references or introduce ambiguity in the metadata (e.g., timestamps or references to unrelated events).
- Ensure all fields (content, comments, and category) are populated. The "category" field must remain unchanged.

Do not give any intro or outro, or explanation etc. All the three are strings like regular text, they should not contain weird start and stop characters.
'''

In [ ]:
import json
import time
from json_repair import repair_json  # Ensure this library is installed

# Batch processing
batch_size = 9  # Adjust the batch size as needed
OUTPUT_FILE = "modified_data.json"
FAILED_FILE = "failed_batches.json"
flag = 0

# List to hold all modified JSON objects
modified_data = []

# List to log irreparable or invalid JSON responses
failed_batches = []

# Start processing from the beginning
for iter, i in enumerate(range(0, len(fdfNoLabel), batch_size)):
    batch_df = fdfNoLabel.iloc[i : i + batch_size]  # Extract batch
    print(f"Processing batch {iter}...")

    # Convert the batch DataFrame to JSON
    batch_json = batch_df.to_json(orient="records")

    # Create prompt for the batch
    prompt = f"""
    Modify the content and comments but not category in each of these articles I provide below.
    Articles (JSON):
    {batch_json}
    """
    prompt = sysInstr + prompt
    if flag == 5:
        client = reAuth()
        flag = 0

    try:
        # Generate content using the model
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
        )
        modified_text = response.choices[0].message.content

        # Parse the modified JSON response
        json_objects = json.loads(modified_text)

        # Validate JSON schema (example: checking required keys)
        if all("content" in obj and "comments" in obj and "category" in obj for obj in json_objects):
            modified_data.extend(json_objects)  # Add valid objects to the list
        else:
            raise ValueError("Response does not match expected schema.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in batch {iter}: {e}")

        # Attempt to repair the JSON if it's malformed
        repaired_text = repair_json(modified_text)
        if repaired_text:
            try:
                json_objects = json.loads(repaired_text)

                # Validate repaired JSON schema
                if all("content" in obj and "comments" in obj and "category" in obj for obj in json_objects):
                    modified_data.extend(json_objects)
                    print(f"Successfully repaired JSON for batch {iter}.")
                else:
                    raise ValueError("Repaired JSON does not match expected schema.")
            except Exception as validation_error:
                print(f"Validation failed for repaired JSON in batch {iter}: {validation_error}")
                failed_batches.append({"batch": iter, "response": repaired_text})
        else:
            failed_batches.append({"batch": iter, "response": modified_text})
    except Exception as e:
        print(f"Unexpected error in batch {iter}: {e}")
        failed_batches.append({"batch": iter, "response": modified_text})

    flag += 1
    time.sleep(0.75)

# Save modified data and failed batches if needed
with open(OUTPUT_FILE, "w") as out_file:
    json.dump(modified_data, out_file, indent=4)

with open(FAILED_FILE, "w") as failed_file:
    json.dump(failed_batches, failed_file, indent=4)

In [ ]:
import json
import time
from json_repair import repair_json  # Ensure this library is installed

# List to hold all modified JSON objects
modified_data = []

# List to log irreparable or invalid JSON responses
failed_batches = []

# Batch processing
batch_size = 9 # Adjust the batch size as needed
start_batch = 61  # Specify the batch number to start from
OUTPUT_FILE = "modified_data.json"
FAILED_FILE = "failed_batches.json"
flag = 0

# Start the loop from the specified batch
for iter, i in enumerate(range(start_batch * batch_size, len(fdfNoLabel), batch_size), start=start_batch):
    batch_df = fdfNoLabel.iloc[i : i + batch_size]  # Extract batch
    print(f"Processing batch {iter}...")

    # Convert the batch DataFrame to JSON
    batch_json = batch_df.to_json(orient="records")

    # Create prompt for the batch
    prompt = f"""
    Modify the content and comments but not category in each of these articles I provide below.
    Articles (JSON):
    {batch_json}
    """
    prompt = sysInstr + prompt
    if flag == 5:
      client = reAuth()
      flag = 0

    try:
        # Generate content using the model
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
        )
        modified_text = response.choices[0].message.content

        # Parse the modified JSON response
        json_objects = json.loads(modified_text)

        # Validate JSON schema (example: checking required keys)
        if all("content" in obj and "comments" in obj and "category" in obj for obj in json_objects):
            modified_data.extend(json_objects)  # Add valid objects to the list
        else:
            raise ValueError("Response does not match expected schema.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in batch {iter}: {e}")

        # Attempt to repair the JSON if it's malformed
        repaired_text = repair_json(modified_text)
        if repaired_text:
            try:
                json_objects = json.loads(repaired_text)

                # Validate repaired JSON schema
                if all("content" in obj and "comments" in obj and "category" in obj for obj in json_objects):
                    modified_data.extend(json_objects)
                    print(f"Successfully repaired JSON for batch {iter}.")
                else:
                    raise ValueError("Repaired JSON does not match expected schema.")
            except Exception as validation_error:
                print(f"Validation failed for repaired JSON in batch {iter}: {validation_error}")
                failed_batches.append({"batch": iter, "response": repaired_text})
        else:
            failed_batches.append({"batch": iter, "response": modified_text})
    except Exception as e:
        print(f"Unexpected error in batch {iter}: {e}")
        failed_batches.append({"batch": iter, "response": modified_text})

    flag += 1
    time.sleep(0.75)

# Save modified data and failed batches if needed
with open(OUTPUT_FILE, "w") as out_file:
    json.dump(modified_data, out_file, indent=4)

with open(FAILED_FILE, "w") as failed_file:
    json.dump(failed_batches, failed_file, indent=4)

Processing batch 61...
Processing batch 62...
Processing batch 63...
Error decoding JSON in batch 63: Expecting value: line 1 column 1 (char 0)
Successfully repaired JSON for batch 63.
Processing batch 64...
Error decoding JSON in batch 64: Expecting value: line 1 column 1 (char 0)
Successfully repaired JSON for batch 64.
Processing batch 65...
Error decoding JSON in batch 65: Expecting value: line 1 column 1 (char 0)
Successfully repaired JSON for batch 65.
Processing batch 66...
Processing batch 67...
Error decoding JSON in batch 67: Expecting value: line 1 column 1 (char 0)
Successfully repaired JSON for batch 67.
Processing batch 68...
Error decoding JSON in batch 68: Expecting value: line 1 column 1 (char 0)
Successfully repaired JSON for batch 68.
Processing batch 69...
Error decoding JSON in batch 69: Expecting value: line 1 column 1 (char 0)
Successfully repaired JSON for batch 69.
Processing batch 70...
Error decoding JSON in batch 70: Expecting value: line 1 column 1 (char 0)

In [ ]:
with open(OUTPUT_FILE, "w") as out_file:
    json.dump(modified_data, out_file, indent=4)

with open(FAILED_FILE, "w") as failed_file:
    json.dump(failed_batches, failed_file, indent=4)

In [ ]:
print(i, iter)
print(modified_data)
print(failed_batches)

1332 148
[{'content': 'A recent radio conversation has sparked controversy after a prominent public figure praised a former US President and questioned the reason behind a pivotal event in American history. The remarks were made during an interview with a well-known radio host on Monday morning. "I mean had this President been a little later, you wouldn\'t have had this event. He was a really tough person, but he had a big heart," the public figure said, despite the fact that the former President was behind a significant historical tragedy. The comments have sent shockwaves on social media, with many users expressing their outrage and disappointment.', 'comments': "Some people have no idea about history.  And if you support them, here's your hat! Others are still hoping this is a dream...a nightmare within a dream of course. Do you even know what you're talking about?", 'category': 'gossipcop'}, {'content': 'A renowned actress is expanding her horizons by launching her own lifestyle br

In [ ]:
import json

# File paths
INPUT_FILE = "/Users/thanikella_nikhil/Downloads/llama_new_temp_modified_data.json"
OUTPUT_FILE = "/Users/thanikella_nikhil/Downloads/llama_new_temp_modified_good_data.json"
BAD_FILE = "bad_before_llama_newData.json"

# Lists to hold cleaned and bad data
cleaned_data = []
bad_data = []

# Load the input file
with open(INPUT_FILE, "r") as file:
    data = json.load(file)

# Process each object in the file
for obj in data:
    # Check if the object has more than 3 fields
    if len(obj.keys()) > 3:
        bad_data.append(obj)
    # Check if the content field is empty
    elif not obj.get("content", "").strip():
        # Skip objects with an empty content field (not added to either list)
        continue
    else:
        cleaned_data.append(obj)

# Save the cleaned data back to the output file
with open(OUTPUT_FILE, "w") as file:
    json.dump(cleaned_data, file, indent=4)

# Save the bad data to a separate file
with open(BAD_FILE, "w") as file:
    json.dump(bad_data, file, indent=4)

print(f"Cleaned data saved to {OUTPUT_FILE}")
print(f"Objects with more than 3 fields saved to {BAD_FILE}")

In [ ]:
import pandas as pd

# Load the JSON array into a pandas DataFrame
json_file = "llama_new_temp_modified_good_data.json"  # The JSON file saved earlier
pkl_file = "llama_new_temp_modified_good_data.pkl"    # The desired .pkl output file


# Read the JSON file
df = pd.read_json(json_file)

# Save the DataFrame as a .pkl file
df.to_pickle(pkl_file)

print(f"JSON data successfully converted to {pkl_file}")

In [ ]:
import pandas as pd

# Load the .pkl file into a pandas DataFrame
llamaTestDf = pd.read_pickle(pkl_file)

llamaTestDf['label'] = 1

dfPath = "data/en/test.pkl"
toDrop = ["content_emotion", "comments_emotion", "emotion_gap", "style_feature"]
temp = pd.read_pickle(dfPath)
df = temp.drop(toDrop, axis=1, inplace=False)
nfdf = df[df['label'] == 0] #True
fdf = df[df['label'] == 1] #False (Fake)
nfdf.head()

In [ ]:
newDf = pd.concat([llamaTestDf, nfdf], ignore_index=True)
print(len(newDf))
newDf.to_pickle("llama_new_temp.pkl")